In [2]:
import pandas as pd

df_sales = pd.read_csv('data/易速鲜花订单记录.csv')
df_sales['消费日期'] = pd.to_datetime(df_sales['消费日期'])
df_sales.head()



,订单号,产品码,消费日期,产品说明,数量,单价,用户码,城市
0,536374,21258,2020-06-01 09:09:00,五彩玫瑰五支装,32,10.95,15100,北京
1,536376,22114,2020-06-01 09:32:00,茉莉花白色25枝,48,3.45,15291,上海
2,536376,21733,2020-06-01 09:32:00,教师节向日葵3枝尤加利5枝,64,2.55,15291,上海
3,536378,22386,2020-06-01 09:37:00,百合粉色10花苞,10,1.95,14688,北京
4,536378,85099C,2020-06-01 09:37:00,橙黄香槟色康乃馨,10,1.95,14688,北京


In [3]:
print('最早消费日期',df_sales['消费日期'].min(), '- 最近消费日期', df_sales['消费日期'].max())

最早消费日期 2020-06-01 09:09:00 - 最近消费日期 2021-06-09 12:31:00


In [4]:
df_sales = df_sales.loc[df_sales['消费日期'] < '2021-06-01'] #只保留整月数据
print('日期范围: %s ~ %s' % (df_sales['消费日期'].min(), df_sales['消费日期'].max())) #显示日期范围

日期范围: 2020-06-01 09:09:00 ~ 2021-05-31 17:39:00


In [5]:

df_sales = df_sales.drop_duplicates()

In [6]:
df_sales_3m = df_sales[(df_sales.消费日期 > '2020-06-01') & (df_sales.消费日期 <= '2020-08-30')]

In [7]:
df_sales_3m.reset_index(drop=True) #重置索引

,订单号,产品码,消费日期,产品说明,数量,单价,用户码,城市
0,536374,21258,2020-06-01 09:09:00,五彩玫瑰五支装,32,10.95,15100,北京
1,536376,22114,2020-06-01 09:32:00,茉莉花白色25枝,48,3.45,15291,上海
2,536376,21733,2020-06-01 09:32:00,教师节向日葵3枝尤加利5枝,64,2.55,15291,上海
3,536378,22386,2020-06-01 09:37:00,百合粉色10花苞,10,1.95,14688,北京
4,536378,85099C,2020-06-01 09:37:00,橙黄香槟色康乃馨,10,1.95,14688,北京
...,...,...,...,...,...,...,...,...
14564,545190,22937,2020-08-29 15:32:00,产品说明掩码,6,18.00,15656,苏州
14565,545190,22722,2020-08-29 15:32:00,产品说明掩码,4,39.50,15656,苏州
14566,545190,22457,2020-08-29 15:32:00,产品说明掩码,60,3.00,15656,苏州
14567,545190,22464,2020-08-29 15:32:00,产品说明掩码,12,25.00,15656,苏州


In [8]:
df_user_LTV = pd.DataFrame(df_sales['用户码'].unique()) #生成以用户码为主键的结构
df_user_LTV.columns = ['用户码'] #设定字段名
df_user_LTV.head() #显示头几行数据


,用户码
0,15100
1,15291
2,14688
3,15311
4,15862


In [9]:
df_R_value = df_sales_3m.groupby('用户码').消费日期.max().reset_index()
df_R_value.columns = ['用户码','最近购买日期']
df_R_value['R值'] = (df_R_value['最近购买日期'].max() - df_R_value['最近购买日期']).dt.days
df_user_LTV = pd.merge(df_user_LTV, df_R_value[['用户码','R值']], on = '用户码') #合并数据
df_user_LTV.head() #显示头几行数据

,用户码,R值
0,15100,45
1,15291,35
2,14688,6
3,15311,5
4,15862,89


In [10]:
df_F_value = df_sales_3m.groupby('用户码').消费日期.count().reset_index()
df_F_value.columns = ['用户码','F值']
df_user_LTV = pd.merge(df_user_LTV, df_F_value[['用户码','F值']], on = '用户码')
df_user_LTV.head()

,用户码,R值,F值
0,15100,45,6
1,15291,35,35
2,14688,6,85
3,15311,5,715
4,15862,89,64


In [11]:
df_sales_3m['总价'] = df_sales_3m['数量'] * df_sales_3m['单价']
df_sales_3m.head()

C:\Users\xin\AppData\Local\Temp\ipykernel_15020\2207970469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales_3m['总价'] = df_sales_3m['数量'] * df_sales_3m['单价']


,订单号,产品码,消费日期,产品说明,数量,单价,用户码,城市,总价
0,536374,21258,2020-06-01 09:09:00,五彩玫瑰五支装,32,10.95,15100,北京,350.4
1,536376,22114,2020-06-01 09:32:00,茉莉花白色25枝,48,3.45,15291,上海,165.6
2,536376,21733,2020-06-01 09:32:00,教师节向日葵3枝尤加利5枝,64,2.55,15291,上海,163.2
3,536378,22386,2020-06-01 09:37:00,百合粉色10花苞,10,1.95,14688,北京,19.5
4,536378,85099C,2020-06-01 09:37:00,橙黄香槟色康乃馨,10,1.95,14688,北京,19.5


In [12]:
df_M_value = df_sales_3m.groupby('用户码').总价.sum().reset_index()
df_M_value.columns = ['用户码','M值']
df_user_LTV = pd.merge(df_user_LTV, df_M_value[['用户码','M值']], on='用户码')
df_user_LTV.head()

,用户码,R值,F值,M值
0,15100,45,6,635.10
1,15291,35,35,1329.95
2,14688,6,85,1472.28
3,15311,5,715,12711.66
4,15862,89,64,354.23


In [13]:
df_sales['总价'] = df_sales['数量'] * df_sales['单价']
df_user_1y = df_sales.groupby('用户码').总价.sum().reset_index()
df_user_1y.columns = ['用户码','年度LTV']
df_user_LTV = pd.merge(df_user_LTV, df_user_1y[['用户码','年度LTV']], on= '用户码')
df_user_LTV

,用户码,R值,F值,M值,年度LTV
0,15100,45,6,635.10,635.10
1,15291,35,35,1329.95,4596.51
2,14688,6,85,1472.28,4449.48
3,15311,5,715,12711.66,58218.04
4,15862,89,64,354.23,659.73
...,...,...,...,...,...
365,15951,1,22,375.17,375.17
366,14745,1,7,240.60,1167.16
367,15724,0,5,103.65,212.30
368,15874,0,5,584.35,4330.67


创建特征集和标签集

In [14]:
X = df_user_LTV.drop(['用户码', '年度LTV'],axis=1)
X.head()


,R值,F值,M值
0,45,6,635.10
1,35,35,1329.95
2,6,85,1472.28
3,5,715,12711.66
4,89,64,354.23


In [15]:
y = df_user_LTV['年度LTV']
y.head()

0      635.10
1     4596.51
2     4449.48
3    58218.04
4      659.73
Name: 年度LTV, dtype: float64

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_rem, y_train, y_rem = train_test_split(X,y,train_size=0.7,random_state=36)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5,random_state = 36)




In [18]:
from sklearn.linear_model import LinearRegression #导入线性回归模型
from sklearn.tree import DecisionTreeRegressor #导入决策树回归模型 
from sklearn.ensemble import RandomForestRegressor #导入随机森林回归模型

model_lr = LinearRegression()
model_dtr = DecisionTreeRegressor()
model_rfr = RandomForestRegressor()

model_lr.fit(X_train, y_train)
model_dtr.fit(X_train, y_train)
model_rfr.fit(X_train, y_train)

RandomForestRegressor()

In [19]:
y_valid_preds_lr = model_lr.predict(X_valid)
y_valid_preds_dtr = model_dtr.predict(X_valid)
y_valid_preds_rfr = model_rfr.predict(X_valid)


In [20]:
X_valid.iloc[2] #随便选择一个数据

R值       1.00
F值     153.00
M值    1413.83
Name: 163, dtype: float64

In [21]:
print('真值:', y_valid.iloc[2])  #真值
print('线性回归预测值:', y_valid_preds_lr[2])  #线性回归模型预测值
print('决策树预测值:', y_valid_preds_dtr[2])  #决策树模型预测值
print('随机森林预测值:', y_valid_preds_rfr[2]) #随机森林模型预测值

真值: 4391.94
线性回归预测值: 7549.228946781513
决策树预测值: 7904.28
随机森林预测值: 4829.942300000002


In [22]:
from sklearn.metrics import r2_score,   median_absolute_error #导入Sklearn评估模块
print('验证集上的R平方分数-线性回归: %0.4f' % r2_score(y_valid, model_lr.predict(X_valid)))
print('验证集上的R平方分数-决策树: %0.4f' % r2_score(y_valid, model_dtr.predict(X_valid)))
print('验证集上的R平方分数-随机森林: %0.4f' % r2_score(y_valid, model_rfr.predict(X_valid)))

验证集上的R平方分数-线性回归: 0.4333
验证集上的R平方分数-决策树: 0.3153
验证集上的R平方分数-随机森林: 0.4955
